# DME DETECTION

In [1]:
import os
import numpy as np
import cv2 # OpenCV
from sklearn.svm import SVC # SVM klasifikator
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.ensemble import RandomForestClassifier #RF klasifikator
import matplotlib.pyplot as plt
import skimage as sk
from skimage import transform
from skimage import util
from skimage import io
import random
from scipy import ndarray
%matplotlib inline

In [2]:
def load_image(path):
    return cv2.imread(path)

def display_image(image):
    plt.imshow(image, 'gray')
    
def image_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

def image_bin(image_gs):
    height, width = image_gs.shape[0:2]
    image_binary = np.ndarray((height, width), dtype=np.uint8)
    ret,image_bin = cv2.threshold(image_gs, 200, 255, cv2.THRESH_BINARY)
    return image_bin

def image_bin_histogram(img):
    ret, image_bin = cv2.threshold(img, 80, 1, cv2.THRESH_BINARY)
    return image_bin

def invert(image):
    return 255-image

def resize_image(image):
    resized_image = sk.transform.resize(image, (100, 100), mode='constant')
    rescaled_image = 255 * resized_image
    final_image = rescaled_image.astype(np.uint8)
    return final_image

def resize_image25(image):
    resized_image = sk.transform.resize(image, (25, 25))
    rescaled_image = 255 * resized_image
    final_image = rescaled_image.astype(np.uint8)
    return final_image

def resize_imageX(image, x):
    resized_image = sk.transform.resize(image, (x, x), mode='constant')
    rescaled_image = 255 * resized_image
    final_image = rescaled_image.astype(np.uint8)
    return final_image

def random_rotation(img):
    random_degree = random.uniform(10, 180)
    return sk.transform.rotate(img, random_degree)

def increase_brightness(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    
    value = random.randint(0, 100)
    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

def reshape_data(input_data):
    nsamples, nx, ny = input_data.shape
    return input_data.reshape((nsamples, nx*ny))

# Učitavanje trening skupa slika

In [3]:
train_dir_dme = 'images/TRAIN/DME'
train_dir_normal = 'images/TRAIN/NORMAL'

nor_imgs = []
dme_imgs = []

folder_path = 'images/edited'

for img_name in os.listdir(train_dir_normal):
    img_path = os.path.join(train_dir_normal, img_name)
    if 'NORMAL' in img_name:
        img = load_image(img_path)
        if img.shape[1] != 512:
            img=img[:,128:640]
        img = img[:, 10:490]
        img2 = (image_bin(image_gray(img)))
        proba, contours, hierarchy = cv2.findContours(img2.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        if (len(contours) != 0) :
            c = max(contours, key = cv2.contourArea)
            x, y, w, h = cv2.boundingRect(c)
            visina = y + h/2
            if (visina > 0 and visina < 60) or (visina > 400 and visina < 496) :
                #cv2.drawContours(img,[c],0,(255,0,0),7)
                cv2.fillPoly(img, pts =[c], color=(0,0,0))
            elif (h == img.shape[0]):
                #cv2.drawContours(img,[c],0,(0,0,255),7)
                cv2.fillPoly(img, pts =[c], color=(0,0,0))
            elif (h > img.shape[0] / 2):
                #cv2.drawContours(img,[c],0,(255,0,255),7)
                cv2.fillPoly(img, pts =[c], color=(0,0,0))
            elif (x == 0 and y == 0):
                #cv2.drawContours(img,[c],0,(255,255,0),7)
                cv2.fillPoly(img, pts =[c], color=(0,0,0))
            #else:
                #cv2.drawContours(img,[c],0,(0,255,0),7)
        nor_imgs.append(img)
        new_file_path = '%s/%s.jpeg' %(folder_path, img_name[:-5])
        img = resize_image(img)
        io.imsave(new_file_path, img)
    
for img_name in os.listdir(train_dir_dme):
    img_path = os.path.join(train_dir_dme, img_name)
    if 'DME' in img_name:
        img = load_image(img_path)
        if img.shape[1] != 512:
            img=img[:,128:640]
        img = img[:, 10:490]
        img2 = (image_bin(image_gray(img)))
        proba2, contours, hierarchy = cv2.findContours(img2.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        if (len(contours) != 0) :
            c = max(contours, key = cv2.contourArea)
            x, y, w, h = cv2.boundingRect(c)
            visina = y + h/2
            if (visina > 0 and visina < 100) or (visina > 350 and visina < 496) :
                #cv2.drawContours(img,[c],0,(255,0,0),7)
                cv2.fillPoly(img, pts =[c], color=(0,0,0))
            elif (h == img.shape[0]):
                #cv2.drawContours(img,[c],0,(0,0,255),7)
                cv2.fillPoly(img, pts =[c], color=(0,0,0))
            elif (h > img.shape[0] / 2):
                #cv2.drawContours(img,[c],0,(255,0,255),7)
                cv2.fillPoly(img, pts =[c], color=(0,0,0))
            elif (x == 0 and y == 0):
                #cv2.drawContours(img,[c],0,(255,255,0),7)
                cv2.fillPoly(img, pts =[c], color=(0,0,0))
            #else:
               # cv2.drawContours(img,[c],0,(0,255,0),7)
        dme_imgs.append(img)
        new_file_path = '%s/%s.jpeg' %(folder_path, img_name[:-5])
        img = resize_image(img)
        io.imsave(new_file_path, img)
   
        
print("Normal images #: ", len(nor_imgs))
print("DME images #: ", len(dme_imgs))

Normal images #:  242
DME images #:  242


# Učitavanje test skupa slika

In [4]:
test_dir_dme = 'images/TEST/DME'
test_dir_normal = 'images/TEST/NORMAL'

nor_imgs_test = []
dme_imgs_test = []

folder_path_test = 'images/edited_test'

for img_name in os.listdir(test_dir_normal):
    img_path = os.path.join(test_dir_normal, img_name)
    if 'NORMAL' in img_name:
        img = load_image(img_path)
        if img.shape[1] != 512:
            img=img[:,128:640]
        img = img[:, 10:490]
        img2 = (image_bin(image_gray(img)))
        proba, contours, hierarchy = cv2.findContours(img2.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        c = max(contours, key = cv2.contourArea)
        x, y, w, h = cv2.boundingRect(c)
        visina = y + h/2
        if (visina > 0 and visina < 60) or (visina > 400 and visina < 496) :
            #cv2.drawContours(img,[c],0,(255,0,0),7)
            cv2.fillPoly(img, pts =[c], color=(0,0,0))
        new_file_path = '%s/%s.jpeg' %(folder_path_test, img_name[:-5])
        img = resize_image(img)
        nor_imgs_test.append(img)
        io.imsave(new_file_path, img)
    
for img_name in os.listdir(test_dir_dme):
    img_path = os.path.join(test_dir_dme, img_name)
    if 'DME' in img_name:
        img = load_image(img_path)
        if img.shape[1] != 512:
            img=img[:,128:640]
        img = img[:, 10:490]
        img2 = (image_bin(image_gray(img)))
        proba2, contours, hierarchy = cv2.findContours(img2.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        c = max(contours, key = cv2.contourArea)
        x, y, w, h = cv2.boundingRect(c)
        visina = y + h/2
        if (visina > 0 and visina < 100) or (visina > 350 and visina < 496) :
            #cv2.drawContours(img,[c],0,(255,0,0),7)
            cv2.fillPoly(img, pts =[c], color=(0,0,0))
        new_file_path = '%s/%s.jpeg' %(folder_path_test, img_name[:-5])
        img = resize_image(img)
        dme_imgs_test.append(img)
        io.imsave(new_file_path, img)
   
        
print("Normal test images #: ", len(nor_imgs_test))
print("DME test images #: ", len(dme_imgs_test))

Normal test images #:  8
DME test images #:  8


# SA AUGMENTACIJOM

# Augmentacija slika

In [5]:
edited_dir = 'images/edited'
augmented_dir = 'images/augmented'

augmented_imgs_nor = []
augmented_imgs_dme = []

num_normal = 0
num_dme = 0

for img_name in os.listdir(edited_dir):
    img_path = os.path.join(edited_dir, img_name)
    img = load_image(img_path)
    
    if 'DME' in img_name:
        num_dme += 1
        augmented_imgs_dme.append(img)
        new_file_path = '%s/DME_%s.jpeg' %(augmented_dir, num_dme)
        io.imsave(new_file_path, img)
    
        num_dme += 1
        rot_image = random_rotation(img)
        new_file_path = '%s/DME_%s.jpeg' %(augmented_dir, num_dme)
        io.imsave(new_file_path, rot_image)
        rot_image = rot_image.astype(np.uint8)
        augmented_imgs_dme.append(rot_image)
        
        num_dme += 1
        bri_image = increase_brightness(img)
        augmented_imgs_dme.append(bri_image)
        new_file_path = '%s/DME_%s.jpeg' %(augmented_dir, num_dme)
        io.imsave(new_file_path, bri_image)
        
        num_dme += 1
        bri_image = increase_brightness(img)
        bri_rot_image = random_rotation(bri_image)
        new_file_path = '%s/DME_%s.jpeg' %(augmented_dir, num_dme)
        io.imsave(new_file_path, bri_rot_image)
        bri_rot_image = bri_rot_image.astype(np.uint8)
        augmented_imgs_dme.append(bri_rot_image)
        
    else :
        num_normal += 1
        augmented_imgs_nor.append(img)
        new_file_path = '%s/NORMAL_%s.jpeg' %(augmented_dir, num_normal)
        io.imsave(new_file_path, img)
    
        num_normal += 1
        rot_image = random_rotation(img)
        new_file_path = '%s/NORMAL_%s.jpeg' %(augmented_dir, num_normal)
        io.imsave(new_file_path, rot_image)
        rot_image = rot_image.astype(np.uint8)
        augmented_imgs_nor.append(rot_image)
        
        num_normal += 1
        bri_image = increase_brightness(img)
        augmented_imgs_nor.append(bri_image)
        new_file_path = '%s/NORMAL_%s.jpeg' %(augmented_dir, num_normal)
        io.imsave(new_file_path, bri_image)
        
        num_normal += 1
        bri_image = increase_brightness(img)
        bri_rot_image = random_rotation(bri_image)
        new_file_path = '%s/NORMAL_%s.jpeg' %(augmented_dir, num_normal)
        io.imsave(new_file_path, bri_rot_image)
        bri_rot_image = bri_rot_image.astype(np.uint8)
        augmented_imgs_nor.append(bri_rot_image)
        
print("Augmented images normal #: ", len(augmented_imgs_nor))
print("Augmented images dme #: ", len(augmented_imgs_dme))

C:\Users\Makaric\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


Augmented images normal #:  968
Augmented images dme #:  968


# HOG deskriptor

In [6]:
nbins = 9 # broj binova
cell_size = (8, 8) # broj piksela po celiji
block_size = (3, 3) # broj celija po bloku

hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1], 
                                  img.shape[0] // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)

# Trening skup

In [7]:
pos_features = []
neg_features = []
labels = []

for img in augmented_imgs_dme:
    pos_features.append(hog.compute(img))
    labels.append(1)

for img in augmented_imgs_nor:
    neg_features.append(hog.compute(img))
    labels.append(0)

pos_features = np.array(pos_features)
neg_features = np.array(neg_features)
x_train = np.vstack((pos_features, neg_features))
y_train = np.array(labels)

x_train = reshape_data(x_train)
print('Train shape HOG: ', x_train.shape, y_train.shape)

Train shape HOG:  (1936, 8100) (1936,)


# Test skup

In [8]:
pos_features_test = []
neg_features_test = []
labels_test = []

for img in dme_imgs_test:
    pos_features_test.append(hog.compute(img))
    labels_test.append(1)

for img in nor_imgs_test:
    neg_features_test.append(hog.compute(img))
    labels_test.append(0)

pos_features_test = np.array(pos_features_test)
neg_features_test = np.array(neg_features_test)
x_test = np.vstack((pos_features_test, neg_features_test))
y_test = np.array(labels_test)

x_test = reshape_data(x_test)
print('Test shape HOG: ', x_test.shape, y_test.shape)

Test shape HOG:  (16, 8100) (16,)


# Obučavanje i validacija SVM klasifikatora

In [9]:
clf_svm = SVC(kernel='linear', probability=True) 
clf_svm.fit(x_train, y_train)
y_train_pred = clf_svm.predict(x_train)
y_test_pred = clf_svm.predict(x_test)
print("Train accuracy HOG+SVM: ", accuracy_score(y_train, y_train_pred))
print("Validation accuracy HOG+SVM: ", accuracy_score(y_test, y_test_pred))

Train accuracy HOG+SVM:  0.8021694214876033
Validation accuracy HOG+SVM:  0.9375


# Obučavanje i validacija KNN klasifikatora

In [10]:
clf_knn = KNeighborsClassifier(n_neighbors=10)
clf_knn = clf_knn.fit(x_train, y_train)
y_train_pred = clf_knn.predict(x_train)
y_test_pred = clf_knn.predict(x_test)
print("Train accuracy HOG+KNN: ", accuracy_score(y_train, y_train_pred))
print("Validation accuracy HOG+KNN: ", accuracy_score(y_test, y_test_pred))

Train accuracy HOG+KNN:  0.734504132231405
Validation accuracy HOG+KNN:  0.6875


# RF klasifikator

In [11]:
clf_rf_hog = RandomForestClassifier(n_estimators=15)
clf_rf_hog = clf_rf_hog.fit(x_train, y_train)
y_train_pred = clf_rf_hog.predict(x_train)
y_test_pred = clf_rf_hog.predict(x_test)
print("Train accuracy HOG+RF: ", accuracy_score(y_train, y_train_pred))
print("Validation accuracy HOG+RF: ", accuracy_score(y_test, y_test_pred))

Train accuracy HOG+RF:  0.8042355371900827
Validation accuracy HOG+RF:  0.9375


# Trening skup histogrami

In [12]:
pos_his_features = []
neg_his_features = []
labels_his = []

for img in augmented_imgs_dme:
    img = resize_imageX(img, 25)
    img = image_gray(img)
    img_bin = image_bin_histogram(img)
    
    sumX = np.sum(img_bin, axis=0)
    sumY = np.sum(img_bin, axis=1)
    sumXY = np.concatenate((sumX, sumY))
    
    pos_his_features.append(sumXY)
    labels_his.append(1)

for img in augmented_imgs_nor:
    img = resize_imageX(img, 25)
    img = image_gray(img)
    img_bin = image_bin_histogram(img)
    
    sumX = np.sum(img_bin, axis=0)
    sumY = np.sum(img_bin, axis=1)
    sumXY = np.concatenate((sumX, sumY))
    
    neg_his_features.append(sumXY)
    labels_his.append(0)

pos_his_features = np.array(pos_his_features)
neg_his_features = np.array(neg_his_features)
x_train_his = np.vstack((pos_his_features, neg_his_features))
y_train_his = np.array(labels_his)

#x_train_his = reshape_data(x_train_his)
print('Train shape histograms: ', x_train_his.shape, y_train_his.shape)

Train shape histograms:  (1936, 50) (1936,)


# Test skup histogrami

In [13]:
pos_his_features_test = []
neg_his_features_test = []
labels_his_test = []

for img in dme_imgs_test:
    img = resize_imageX(img, 25)
    img = image_gray(img)
    img_bin = image_bin_histogram(img)
    
    sumX = np.sum(img_bin, axis=0)
    sumY = np.sum(img_bin, axis=1)
    sumXY = np.concatenate((sumX, sumY))
    
    pos_his_features_test.append(sumXY)
    labels_his_test.append(1)

for img in nor_imgs_test:
    img = resize_imageX(img, 25)
    img = image_gray(img)
    img_bin = image_bin_histogram(img)
    
    sumX = np.sum(img_bin, axis=0)
    sumY = np.sum(img_bin, axis=1)
    sumXY = np.concatenate((sumX, sumY))
    
    neg_his_features_test.append(sumXY)
    labels_his_test.append(0)

pos_his_features_test = np.array(pos_his_features_test)
neg_his_features_test = np.array(neg_his_features_test)
x_test_his = np.vstack((pos_his_features_test, neg_his_features_test))
y_test_his = np.array(labels_his_test)

#x_test_his = reshape_data(x_test_his)
print('Test shape histograms: ', x_test_his.shape, y_test_his.shape)

Test shape histograms:  (16, 50) (16,)


# Histogrami SVM

In [33]:
clf_svm_his = SVC(kernel='linear', probability=True) 
clf_svm_his.fit(x_train_his, y_train_his)
y_train_pred_his = clf_svm_his.predict(x_train_his)
y_test_pred_his = clf_svm_his.predict(x_test_his)
print("Train accuracy histograms+SVM: ", accuracy_score(y_train_his, y_train_pred_his))
print("Validation accuracy hisrograms+SVM: ", accuracy_score(y_test_his, y_test_pred_his))

Train accuracy histograms+SVM:  0.5986570247933884
Validation accuracy hisrograms+SVM:  0.625


# Histogrami KNN

In [15]:
clf_knn_his = KNeighborsClassifier(n_neighbors=17)
clf_knn_his = clf_knn_his.fit(x_train_his, y_train_his)
y_train_pred_his = clf_knn_his.predict(x_train_his)
y_test_pred_his = clf_knn_his.predict(x_test_his)
print("Train accuracy histograms+KNN: ", accuracy_score(y_train_his, y_train_pred_his))
print("Validation accuracy histograms+KNN: ", accuracy_score(y_test_his, y_test_pred_his))

Train accuracy histograms+KNN:  0.6100206611570248
Validation accuracy histograms+KNN:  0.8125


# Histogrami RF

In [16]:
clf_rf_his = RandomForestClassifier(n_estimators=15)
clf_rf_his = clf_rf_his.fit(x_train_his, y_train_his)
y_train_pred_his = clf_rf_his.predict(x_train_his)
y_test_pred_his = clf_rf_his.predict(x_test_his)
print("Train accuracy histograms+RF: ", accuracy_score(y_train_his, y_train_pred_his))
print("Validation accuracy histograms+RF: ", accuracy_score(y_test_his, y_test_pred_his))
 

Train accuracy histograms+RF:  0.7241735537190083
Validation accuracy histograms+RF:  0.6875


#  BEZ AUGMENTACIJE

# HOG

# Trening skup

In [17]:
edited_dir = 'images/edited'
pos_features_bezA = []
neg_features_bezA = []
labels_bezA = []

for img_name in os.listdir(edited_dir):
    img_path = os.path.join(edited_dir, img_name)
    img = load_image(img_path)
    
    if 'DME' in img_name:
        pos_features_bezA.append(hog.compute(img))
        labels_bezA.append(1)
    else :
        neg_features_bezA.append(hog.compute(img))
        labels_bezA.append(0)
        
pos_features_bezA = np.array(pos_features_bezA)
neg_features_bezA = np.array(neg_features_bezA)
x_train_bezA = np.vstack((pos_features_bezA, neg_features_bezA))
y_train_bezA = np.array(labels_bezA)

x_train_bezA = reshape_data(x_train_bezA)
print('Train shape HOG without augmentation: ', x_train_bezA.shape, y_train_bezA.shape)

Train shape HOG without augmentation:  (484, 8100) (484,)


Test skup je isti 

# Obučavanje i validacija SVM klasifikatora

In [18]:
clf_svm_bezA = SVC(kernel='linear', probability=True) 
clf_svm_bezA.fit(x_train_bezA, y_train_bezA)
y_train_pred_bezA = clf_svm_bezA.predict(x_train_bezA)
y_test_pred_bezA = clf_svm_bezA.predict(x_test)
print("Train accuracy HOG+SVM without augmentation: ", accuracy_score(y_train_bezA, y_train_pred_bezA))
print("Validation accuracy HOG+SVM without augmentation: ", accuracy_score(y_test, y_test_pred_bezA))

Train accuracy HOG+SVM without augmentation:  1.0
Validation accuracy HOG+SVM without augmentation:  0.9375


# Obučavanje i validacija KNN klasifikatora

In [30]:
clf_knn_bezA = KNeighborsClassifier(n_neighbors=10)
clf_knn_bezA = clf_knn_bezA.fit(x_train_bezA, y_train_bezA)
y_train_pred_bezA = clf_knn_bezA.predict(x_train_bezA)
y_test_pred_bezA = clf_knn_bezA.predict(x_test)
print("Train accuracy HOG+KNN without augmentation: ", accuracy_score(y_train_bezA, y_train_pred_bezA))
print("Validation accuracy HOG+KNN without augmentation: ", accuracy_score(y_test, y_test_pred_bezA))

Train accuracy HOG+KNN without augmentation:  0.8409090909090909
Validation accuracy HOG+KNN without augmentation:  0.6875


# RF klasifikator

In [29]:
clf_rf_hog_bezA = RandomForestClassifier(n_estimators=15)
clf_rf_hog_bezA = clf_rf_hog_bezA.fit(x_train_bezA, y_train_bezA)
y_train_pred_bezA = clf_rf_hog_bezA.predict(x_train_bezA)
y_test_pred_bezA = clf_rf_hog_bezA.predict(x_test)
print("Train accuracy HOG+RF without augmentation: ", accuracy_score(y_train_bezA, y_train_pred_bezA))
print("Validation accuracy HOG+RF without augmentation: ", accuracy_score(y_test, y_test_pred_bezA))

Train accuracy HOG+RF without augmentation:  0.9979338842975206
Validation accuracy HOG+RF without augmentation:  1.0


# HISTOGRAMI

# Trening skup

In [21]:
edited_dir = 'images/edited'
pos_his_features_bezA = []
neg_his_features_bezA = []
labels_his_bezA = []

for img_name in os.listdir(edited_dir):
    img_path = os.path.join(edited_dir, img_name)
    img = load_image(img_path)
    
    if 'DME' in img_name:
        img = resize_imageX(img, 25)
        img = image_gray(img)
        img_bin = image_bin_histogram(img)
    
        sumX = np.sum(img_bin, axis=0)
        sumY = np.sum(img_bin, axis=1)
        sumXY = np.concatenate((sumX, sumY))
    
        pos_his_features_bezA.append(sumXY)
        labels_his_bezA.append(1)
    else:
        img = resize_imageX(img, 25)
        img = image_gray(img)
        img_bin = image_bin_histogram(img)
    
        sumX = np.sum(img_bin, axis=0)
        sumY = np.sum(img_bin, axis=1)
        sumXY = np.concatenate((sumX, sumY))
    
        neg_his_features_bezA.append(sumXY)
        labels_his_bezA.append(0)

pos_his_features_bezA = np.array(pos_his_features_bezA)
neg_his_features_bezA = np.array(neg_his_features_bezA)
x_train_his_bezA = np.vstack((pos_his_features_bezA, neg_his_features_bezA))
y_train_his_bezA = np.array(labels_his_bezA)

#x_train_his = reshape_data(x_train_his)
print('Train shape histograms without augmentation: ', x_train_his_bezA.shape, y_train_his_bezA.shape)

Train shape histograms without augmentation:  (484, 50) (484,)


Test skup je isti 

# Histogrami SVM

In [22]:
clf_svm_his_bezA = SVC(kernel='linear', probability=True) 
clf_svm_his_bezA.fit(x_train_his_bezA, y_train_his_bezA)
y_train_pred_his_bezA = clf_svm_his_bezA.predict(x_train_his_bezA)
y_test_pred_his_bezA = clf_svm_his_bezA.predict(x_test_his)
print("Train accuracy histograms+SVM without augmentation: ", accuracy_score(y_train_his_bezA, y_train_pred_his_bezA))
print("Validation accuracy hisrograms+SVM without augmentation: ", accuracy_score(y_test_his, y_test_pred_his_bezA))

Train accuracy histograms+SVM without augmentation:  0.731404958677686
Validation accuracy hisrograms+SVM without augmentation:  0.625


# Histogrami KNN 

In [23]:
clf_knn_his_bezA = KNeighborsClassifier(n_neighbors=17)
clf_knn_his_bezA = clf_knn_his_bezA.fit(x_train_his_bezA, y_train_his_bezA)
y_train_pred_his_bezA = clf_knn_his_bezA.predict(x_train_his_bezA)
y_test_pred_his_bezA = clf_knn_his_bezA.predict(x_test_his)
print("Train accuracy histograms+KNN without augmentation: ", accuracy_score(y_train_his_bezA, y_train_pred_his_bezA))
print("Validation accuracy histograms+KNN without augmentation: ", accuracy_score(y_test_his, y_test_pred_his_bezA))

Train accuracy histograms+KNN without augmentation:  0.75
Validation accuracy histograms+KNN without augmentation:  0.75


# Histogrami RF

In [27]:
clf_rf_his_bezA = RandomForestClassifier(n_estimators=15)
clf_rf_his_bezA = clf_rf_his_bezA.fit(x_train_his_bezA, y_train_his_bezA)
y_train_pred_his_bezA = clf_rf_his_bezA.predict(x_train_his_bezA)
y_test_pred_his_bezA = clf_rf_his_bezA.predict(x_test_his)
print("Train accuracy histograms+RF without augmentation: ", accuracy_score(y_train_his_bezA, y_train_pred_his_bezA))
print("Validation accuracy histograms+RF without augmentation: ", accuracy_score(y_test_his, y_test_pred_his_bezA))
 

Train accuracy histograms+RF without augmentation:  0.993801652892562
Validation accuracy histograms+RF without augmentation:  0.75
